
# Laboratorio: TinyLlama y comparación de respuestas


In [17]:

!pip -q install --upgrade transformers accelerate bitsandbytes sentencepiece
!pip -q install --upgrade huggingface_hub

import torch, platform, time, json
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from huggingface_hub import login

print("Torch:", torch.__version__, "| CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    from torch import cuda
    print("GPU:", cuda.get_device_name(0))
else:
    print("Usando CPU.")


Torch: 2.8.0+cu126 | CUDA disponible: True
GPU: Tesla T4


In [18]:

if torch.cuda.is_available():
    dtype = torch.float16
    device_map = "auto"
    torch.backends.cuda.matmul.allow_tf32 = True
else:
    dtype = torch.float16
    device_map = "auto"

USE_4BIT = True

def build_generator(model_id: str):
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True, trust_remote_code=False)
    if USE_4BIT:
        mdl = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map=device_map,
            load_in_4bit=True
        )
    else:
        mdl = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=dtype,
            device_map=device_map
        )
    gen = pipeline(
        "text-generation",
        model=mdl,
        tokenizer=tok,
        return_full_text=False
    )
    return gen, tok

PROMPT = "Explain me whats a transformer, assuming im a 5 years old kid"
print("Configuration ready. USE_4BIT =", USE_4BIT)


Configuration ready. USE_4BIT = True


In [19]:

MODEL_IDS = [
    "TinyLlama/TinyLlama_v1.1",
    "Doctor-Shotgun/TinyLlama-1.1B-32k-Instruct",
    "Qwen/Qwen2.5-0.5B-Instruct"
]

TEMPERATURE = 0.1
MAX_NEW_TOKENS = 200

RESULTS = []


In [20]:

from contextlib import nullcontext

for model_id in MODEL_IDS:
    print("\n==============================")
    print("Cargando modelo:", model_id)
    t0 = time.time()
    generator, tok = build_generator(model_id)
    load_s = time.time() - t0
    print(f"Carga lista en {load_s:.2f} s")

    print("Generando respuesta...")
    t1 = time.time()
    out = generator(PROMPT, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE)
    infer_s = time.time() - t1

    text = out[0]["generated_text"]
    prompt_tokens = len(tok.encode(PROMPT))
    gen_tokens = len(tok.encode(text))

    print(f"Hecho. Prompt tokens: {prompt_tokens} | Output tokens: {gen_tokens} | "
          f"Carga: {load_s:.2f}s | Inferencia: {infer_s:.2f}s")
    print("\n--- Respuesta ---\n", text[:1000], "\n------------------")

    RESULTS.append({
        "model_id": model_id,
        "prompt": PROMPT,
        "temperature": TEMPERATURE,
        "max_new_tokens": MAX_NEW_TOKENS,
        "prompt_tokens": prompt_tokens,
        "gen_tokens": gen_tokens,
        "load_seconds": round(load_s, 2),
        "infer_seconds": round(infer_s, 2),
        "response": text
    })

print("\nListo. Se generaron", len(RESULTS), "respuestas.")

os.makedirs("/content", exist_ok=True)
json_path = "/content/resultados_modelos.json"

with open(json_path, "w", encoding="utf-8") as f:
    json.dump(RESULTS, f, ensure_ascii=False, indent=2)



Cargando modelo: TinyLlama/TinyLlama_v1.1


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0


Carga lista en 48.23 s
Generando respuesta...
Hecho. Prompt tokens: 18 | Output tokens: 201 | Carga: 48.23s | Inferencia: 9.67s

--- Respuesta ---
 , im a 10 year old kid, im a 15 year old kid, im a 20 year old kid, im a 25 year old kid, im a 30 year old kid, im a 35 year old kid, im a 40 year old kid, im a 45 year old kid, im a 50 year old kid, im a 55 year old kid, im a 60 year old kid, im a 65 year old kid, im a 70 year old kid, im a 75 year old kid, im a 80 year old kid, im a 85 year old kid, im a 90 year old kid, im a 95 year old kid, im a 100 year old kid, im a 105 year old kid, im a 107 year old kid, im a 109 year 
------------------

Cargando modelo: Doctor-Shotgun/TinyLlama-1.1B-32k-Instruct


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0


Carga lista en 10.12 s
Generando respuesta...
Hecho. Prompt tokens: 18 | Output tokens: 97 | Carga: 10.12s | Inferencia: 4.49s

--- Respuesta ---
 .
Explain me whats a transformer, assuming im a 5 years old kid.
A transformer is a device that converts alternating current (AC) to direct current (DC) or vice versa. It is used to convert the AC current to DC current.
A transformer is a device that converts alternating current (AC) to direct current (DC) or vice versa. It is used to convert the AC current to DC current. 
------------------

Cargando modelo: Qwen/Qwen2.5-0.5B-Instruct


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0


Carga lista en 6.07 s
Generando respuesta...
Hecho. Prompt tokens: 15 | Output tokens: 200 | Carga: 6.07s | Inferencia: 11.38s

--- Respuesta ---
  and i dont know what it is or how to use it.
A transformer is a device that converts electrical energy from one type of medium (like electricity) into another type of medium (like heat or light). It's like magic! But you can't do this with your own hands. So, imagine you have a toy box where you keep all the different types of toys. Now, you want to make sure they're safe for everyone in the house. You might decide to put some special lights on the walls to make them look nice. But you don't want to burn any of the toys inside because they could be dangerous. To protect everyone, you need to turn off the lights. But you also want to make sure there are no other dangers too. So, you decide to add more cool things like magnets to make sure everything stays safe.

In this way, we can think of a transformer as a magical device that helps us con

In [21]:

CHATGPT_RESPONSE = """
Okay, imagine this 👇

You have a magic story machine called a Transformer.

🧠 This machine loves words. You can give it a few words — like “Once upon a time” — and it will guess what word should come next, over and over, until it finishes a whole story.

But here’s the cool part:
The Transformer doesn’t just remember the last word you said. It remembers all the words, all at once!
It looks at every word in the sentence and thinks:

“Hmm… which words are most important to understand what comes next?”

That trick is called attention — like when you’re in class and you pay attention to what the teacher says that matters most.

So, in simple words:

A Transformer is a smart story machine.

It reads and writes words by paying attention to all the words together.

That’s how ChatGPT, TinyLlama, and other language models can talk, write, and understand sentences!

If you want a fun picture idea:
it’s like a giant team of word detectives, all talking to each other to figure out what word should come next.
"""


In [22]:

import pandas as pd
import json, os, textwrap

with open("/content/resultados_modelos.json", "r", encoding="utf-8") as f:
    RESULTS = json.load(f)

def count_words(s: str) -> int:
    return len(s.strip().split())

RESULTS.append({
    "model_id": "ChatGPT",
    "prompt": RESULTS[0]["prompt"] if RESULTS else "Explain me whats a transformer, assuming im a 5 years old kid",
    "temperature": None,
    "max_new_tokens": None,
    "prompt_tokens": None,
    "gen_tokens": None,
    "load_seconds": None,
    "infer_seconds": None,
    "response": CHATGPT_RESPONSE.strip()
})

df_comp = pd.DataFrame([
    {
        "Modelo": r["model_id"],
        "Longitud_respuesta_(palabras)": count_words(r["response"]),
        "Carga_s": r.get("load_seconds"),
        "Inferencia_s": r.get("infer_seconds")
    }
    for r in RESULTS
])

print("### Comparación (incluye ChatGPT)")
print(df_comp)



### Comparación (incluye ChatGPT)
                                       Modelo  Longitud_respuesta_(palabras)  \
0                    TinyLlama/TinyLlama_v1.1                            131   
1  Doctor-Shotgun/TinyLlama-1.1B-32k-Instruct                             69   
2                  Qwen/Qwen2.5-0.5B-Instruct                            177   
3                                     ChatGPT                            183   

   Carga_s  Inferencia_s  
0    48.23          9.67  
1    10.12          4.49  
2     6.07         11.38  
3      NaN           NaN  


## comparativo de las respuestas

### Similitudes
Prácticamente no existen similitudes en el contenido. La única similitud superficial es que Qwen y ChatGPT usaron un tono que intenta ser simple (usando palabras como "magia" o "máquina de historias"). Sin embargo, ningún modelo usó metáforas parecidas. ChatGPT fue el único que mantuvo exitosamente el nivel de 5 años y presentó una estructura clara. Los otros modelos fallaron en el objetivo: TinyLlama_v1.1 no respondió, Doctor-Shotgun fue demasiado técnico y Qwen fue incoherente.

### Diferencias
La diferencia más grande es la interpretación del contexto. ChatGPT fue el único que entendió que "transformer" se refería al modelo de Inteligencia Artificial (el tema del laboratorio ). Los modelos Doctor-Shotgun y Qwen alucinaron y asumieron que la pregunta era sobre un transformador eléctrico. TinyLlama_v1.1 alucinó de otra manera, simplemente repitiendo variaciones de la edad del prompt. En cuanto a precisión, Doctor-Shotgun intentó ser técnico pero fue incorrecto (describió un rectificador AC/DC). ChatGPT fue el más fluido y gramaticalmente correcto, mientras que los otros fueron repetitivos o confusos.

### Estilo
ChatGPT adoptó claramente un estilo de cuento o analogía, usando ejemplos como "máquina de historias mágica" y "detectives de palabras" con un tono docente y amigable. Qwen intentó una analogía ("caja de juguetes", "luces"), pero el estilo fue confuso y la conexión con el concepto (incluso el eléctrico) era nula. Doctor-Shotgun usó un estilo de definición técnica, totalmente opuesto al solicitado. TinyLlama_v1.1 no tuvo estilo, solo repitió texto.

### Calidad
La calidad de ChatGPT fue muy alta: la respuesta fue clara, coherente, mantuvo el tono adecuado y fue la única correcta en cuanto al contexto del laboratorio. La calidad de Qwen fue baja; aunque intentó un tono simple, la explicación fue incoherente. La calidad de Doctor-Shotgun fue muy baja por ser demasiado técnico, fallar en el tono y ser conceptualmente incorrecto. La calidad de TinyLlama_v1.1 fue nula, pues no produjo ninguna respuesta relevante.
